In [5]:
import os
import sys
import numpy as np
import pandas as pd

model01 = pd.read_csv("../model01/oof/oof.csv").rename(columns={"sii": "sii01"})
model02 = pd.read_csv("../model02/oof/oof.csv").rename(columns={"sii": "sii02"})
model03 = pd.read_csv("../model03/oof/oof.csv").rename(columns={"pred_sii": "sii03"})

gt = pd.read_csv("../../../inputs/train.csv")
gt = gt[["id", "sii"]].dropna()

gt = gt.merge(model01, on="id").merge(model02, on="id").merge(model03, on="id")
gt

,id,sii,sii01,sii02,sii03
0,00008ff9,2.0,0.451007,0.451007,0.325819
1,000fd460,0.0,0.182999,0.182999,0.275750
2,00105258,0.0,0.556960,0.556960,1.044493
3,00115b9f,1.0,0.084310,0.084310,0.413800
4,001f3379,1.0,0.419265,0.419265,0.357467
...,...,...,...,...,...
2731,ff6c2bb8,0.0,0.425105,0.425105,0.547489
2732,ff759544,1.0,0.134415,0.134415,0.234311
2733,ff8a2de4,1.0,0.782677,0.782677,0.969899
2734,ffcd4dbd,1.0,0.702262,0.702262,0.943198


In [8]:
from sklearn.metrics import *
from scipy.optimize import minimize


def quadratic_weighted_kappa(y_true, y_pred):
    return cohen_kappa_score(y_true, y_pred, weights="quadratic")


def threshold_Rounder(oof_non_rounded, thresholds):
    return np.where(
        oof_non_rounded < thresholds[0],
        0,
        np.where(
            oof_non_rounded < thresholds[1],
            1,
            np.where(oof_non_rounded < thresholds[2], 2, 3),
        ),
    )


def evaluate_predictions(thresholds, y_true, oof_non_rounded):
    mean_weight = thresholds[:3]
    thresholds = thresholds[3:]

    model01_pred = oof_non_rounded[:, 0]
    model02_pred = oof_non_rounded[:, 1]
    model03_pred = oof_non_rounded[:, 2]
    oof_non_rounded = (
        mean_weight[0] * model01_pred
        + mean_weight[1] * model02_pred
        + mean_weight[2] * model03_pred
    )
    rounded_p = threshold_Rounder(oof_non_rounded, thresholds)
    return -quadratic_weighted_kappa(y_true, rounded_p)


KappaOPtimizer = minimize(
    evaluate_predictions,
    x0=[0.333, 0.333, 0.333, 0.5, 1.5, 2.5],
    args=(gt["sii"], gt[["sii01", "sii02", "sii03"]].values),
    method="Nelder-Mead",
)

pred = gt[["sii01", "sii02", "sii03"]].values
pred = (
    pred[:, 0] * KappaOPtimizer.x[0]
    + pred[:, 1] * KappaOPtimizer.x[1]
    + pred[:, 2] * KappaOPtimizer.x[2]
)

oof_tuned = threshold_Rounder(pred, KappaOPtimizer.x[3:])
tKappa = quadratic_weighted_kappa(gt["sii"], oof_tuned)

print(f"tuned Kappa: {tKappa:.3f}")

tuned Kappa: 0.454


In [12]:
# 平均値

pred = gt[["sii01", "sii02", "sii03"]].values
pred = pred[:, 0] * (1 / 3) + pred[:, 1] * (1 / 3) + pred[:, 2] * (1 / 3)

oof_tuned = threshold_Rounder(pred, KappaOPtimizer.x[3:])
tKappa = quadratic_weighted_kappa(gt["sii"], oof_tuned)

print(f"tuned Kappa: {tKappa:.3f}")

tuned Kappa: 0.448
